In [1]:
# ! pip install git+https://github.com/jjshoots/bsuite.git@f8d81b2
from bsuite import sweep
import bsuite
import os
from rllib_wrapper import RLLibEnv
from ray.rllib.algorithms.ppo import PPOConfig
from ray import air
from ray import tune
from ray.tune.registry import register_env

In [24]:
from bsuite import sweep
envs = []
for i in range(0,len(sweep.SWEEP),1):
    if sweep.SWEEP[i].split('/')[0] not in envs:
        #if not 'scale' in sweep.SWEEP[i].split('/')[0]:
        envs.append(sweep.SWEEP[i].split('/')[0])
env_strings = []
for s in ['1', '5', '10', '15']:
    for e in envs:
        env_strings.append(e+'/'+s)
print(len(env_strings))

92


In [3]:
path = '/Users/adamprice/RLOlympics/results/PPO/data.db'
env_str = 'cartpole_swingup/1'
if not os.path.exists(path):
   os.makedirs(path)

env = bsuite.load_from_id(env_str)

def env_creator(env_config):
    env = bsuite.load_and_record_to_sqlite(env_str, db_path=path)
    env = bsuite.load_from_id(env_str)
    return RLLibEnv(env)

import ray
ray.rllib.utils.check_env(RLLibEnv(env))

register_env(name=env_str.replace('/','_'), env_creator=env_creator)

2023-07-18 12:14:00,061	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.


Loaded bsuite_id: cartpole_swingup/1.


In [11]:
config = PPOConfig()
config.training(lr=tune.grid_search([.01,0.001,0.0005]), num_sgd_iter=3, train_batch_size=tune.grid_search([64,128,256,512]), entropy_coeff=0.001, sgd_minibatch_size=64, lambda_=0.95,
                model={"fcnet_hiddens": [32], "fcnet_activation": "tanh",})
config.resources(num_cpus_per_worker=1)
config.rollouts(num_rollout_workers=0)
#config.exploration(explore=True, exploration_config={"type": "RE3", "embeds_dim": 32, "beta_schedule": "constant", "sub_exploration": {"type": "StochasticSampling",},})
config.exploration(explore=True, exploration_config={   
                    "type": "Curiosity",
                    "eta": 0.2,
                    "lr": 0.001,
                    "feature_dim": 64,
                    "feature_net_config": {
                        "fcnet_hiddens": [],
                        "fcnet_activation": "relu",
                    },
                    "sub_exploration": {
                        "type": "StochasticSampling",
                    }},)
config.framework('torch')

config.environment(env=env_str.replace('/','_'))  

tune.Tuner(
        "PPO",
        run_config=air.RunConfig(stop={"episodes_total": env.bsuite_num_episodes}, verbose=2),
        param_space=config.to_dict(),
        ).fit()

(PPO pid=95638) 2023-07-18 13:44:47,847	WARNING algorithm_config.py:643 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported. [repeated 9x across cluster]
(PPO pid=94275) 2023-07-18 13:40:41,161	INFO algorithm.py:536 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags. [repeated 9x across cluster]
(PPO pid=94275) 2023-07-18 13:40:41,173	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset. [repeated 9x across cluster]
(PPO pid=94275) 2023-07-18 13:40:41,291	WARNING util.py:68 -- Install gputil for GPU system monitoring. [repeated 9x across cluster]


(PPO pid=95639) Loaded bsuite_id: cartpole_swingup/1. [repeated 9x across cluster]
(PPO pid=95639) Logging results to SQLite database in /Users/adamprice/RLOlympics/results/PPO/data.db. [repeated 9x across cluster]


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_cartpole_swingup_1_d3aa5_00000,3840,"{'ObsPreprocessorConnector_ms': 0.016121069590250652, 'StateBufferConnector_ms': 0.009250640869140625, 'ViewRequirementAgentConnector_ms': 0.24902423222859701}","{'num_env_steps_sampled': 3840, 'num_env_steps_trained': 3840, 'num_agent_steps_sampled': 3840, 'num_agent_steps_trained': 3840}",{},482.833,{},0,-5.45,-17.3,0,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.11014328710734844, 'cur_kl_coeff': 1.1102230246251568e-17, 'cur_lr': 0.01, 'total_loss': 0.00020812731236219406, 'policy_loss': 2.146698534488678e-06, 'vf_loss': 0.00020641650462494, 'vf_explained_var': -0.7508742809295654, 'kl': 8.762519196163036e-07, 'entropy': 0.0004409714504921188, 'entropy_coeff': 0.001}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 64.0, 'num_grad_updates_lifetime': 179.0, 'diff_num_grad_updates_vs_sampler_policy': 1.0}}, 'num_env_steps_sampled': 3840, 'num_env_steps_trained': 3840, 'num_agent_steps_sampled': 3840, 'num_agent_steps_trained': 3840}",3840,3840,3840,64,182.326,3840,64,182.326,0,0,0,0,64,"{'cpu_util_percent': 73.9, 'ram_util_percent': 56.8}",{},{},{},"{'mean_raw_obs_processing_ms': 0.9661069735963624, 'mean_inference_ms': 3.2034059297329525, 'mean_action_processing_ms': 0.309174309476905, 'mean_env_wait_ms': 0.25387105184804076, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 0.0, 'episode_reward_min': -17.299999999999976, 'episode_reward_mean': -5.449999999999995, 'episode_len_mean': 482.8333333333333, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-17.299999999999976, -7.19999999999999, -3.800000000000002, -3.1000000000000014, -1.3, 0.0], 'episode_lengths': [262, 179, 177, 277, 1001, 1001]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.9661069735963624, 'mean_inference_ms': 3.2034059297329525, 'mean_action_processing_ms': 0.309174309476905, 'mean_env_wait_ms': 0.25387105184804076, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.016121069590250652, 'StateBufferConnector_ms': 0.009250640869140625, 'ViewRequirementAgentConnector_ms': 0.24902423222859701}}","{'training_iteration_time_ms': 245.463, 'sample_time_ms': 223.605, 'load_time_ms': 0.399, 'load_throughput': 160365.288, 'learn_time_ms': 21.04, 'learn_throughput': 3041.762, 'synch_weights_time_ms': 0.005}"
PPO_cartpole_swingup_1_d3aa5_00001,68288,"{'ObsPreprocessorConnector_ms': 0.010343910991281702, 'StateBufferConnector_ms': 0.007828422214673914, 'ViewRequirementAgentConnector_ms': 0.20373904186746347}","{'num_env_steps_sampled': 68288, 'num_env_steps_trained': 68288, 'num_agent_steps_sampled': 68288, 'num_agent_steps_trained': 68288}",{},982.71,{},0,-2.58261,-57.7,0,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.010231672165294489, 'cur_kl_coeff': 2.5e-322, 'cur_lr': 0.001, 'total_loss': 1.1237959067026775e-06, 'policy_loss': 5.463759104410807e-08, 'vf_loss': 1.5982331357614992e-06, 'vf_explained_var': -0.5059420267740885, 'kl': 6.6617400849150465e-09, 'entropy': 0.0005314586063226064, 'entropy_coeff': 0.001}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 64.0, 'num_grad_updates_lifet

2023-07-18 13:44:54,174	ERROR tune_controller.py:873 -- Trial task failed for trial PPO_cartpole_swingup_1_d3aa5_00007
Traceback (most recent call last):
  File "/Users/adamprice/Applications/anaconda3/envs/rlolympics/lib/python3.9/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/Users/adamprice/Applications/anaconda3/envs/rlolympics/lib/python3.9/site-packages/ray/_private/auto_init_hook.py", line 18, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/Users/adamprice/Applications/anaconda3/envs/rlolympics/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/Users/adamprice/Applications/anaconda3/envs/rlolympics/lib/python3.9/site-packages/ray/_private/worker.py", line 2540, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::PPO.train() (pid=95636, ip=127.0.0.1, actor_id=5a8bbc6fd04d5b2

ResultGrid<[
  Result(
    error='RayTaskError(RuntimeError)',
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.11014328710734844, 'cur_kl_coeff': 1.1102230246251568e-17, 'cur_lr': 0.01, 'total_loss': 0.00020812731236219406, 'policy_loss': 2.146698534488678e-06, 'vf_loss': 0.00020641650462494, 'vf_explained_var': -0.7508742809295654, 'kl': 8.762519196163036e-07, 'entropy': 0.0004409714504921188, 'entropy_coeff': 0.001}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 64.0, 'num_grad_updates_lifetime': 179.0, 'diff_num_grad_updates_vs_sampler_policy': 1.0}}, 'num_env_steps_sampled': 3840, 'num_env_steps_trained': 3840, 'num_agent_steps_sampled': 3840, 'num_agent_steps_trained': 3840}, 'sampler_results': {'episode_reward_max': 0.0, 'episode_reward_min': -17.299999999999976, 'episode_reward_mean': -5.449999999999995, 'episode_len_mean': 482.8333333333333, 'episode_media

In [ ]:

config = DQNConfig()
print(config.exploration_config)  # doctest: +SKIP
        # >>> explore_config = config.exploration_config.update( # doctest: +SKIP
        # ...     {
        # ...         "initial_epsilon": 1.5,
        # ...         "final_epsilon": 0.01,
        # ...         "epsilone_timesteps": 5000,
        # ...     }

{'type': 'EpsilonGreedy', 'initial_epsilon': 1.0, 'final_epsilon': 0.02, 'epsilon_timesteps': 10000}


In [ ]:
from ray.rllib.algorithms.dqn import DQNConfig
config = DQNConfig()
config.training(train_batch_size=8,
                model={"fcnet_hiddens": [32, 32], "fcnet_activation": "tanh",})
config.resources(num_cpus_per_worker=1)
config.rollouts(num_rollout_workers=0)
config.exploration(explore=True, exploration_config={"type": "SoftQ", "temperature": 1.0})
config.framework('torch')

config.environment(env=env_str.replace('/','_'))  

tune.Tuner(
        "DQN",
        run_config=air.RunConfig(stop={"episodes_total": env.bsuite_num_episodes}, verbose=2),
        param_space=config.to_dict(),
        ).fit()

2023-07-17 15:49:27,882	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/a3c.py` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
2023-07-17 15:49:27,883	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/a3c.py` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
2023-07-17 15:49:28,016	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/maml/maml.py` has been deprecated. Use `rllib_contrib/maml/` instead. This will raise an error in the future!
2023-07-17 15:49:31,645	INFO worker.py:1636 -- Started a local Ray instance.
2023-07-17 15:49:33,567	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2023-07-17 15:49:33,572	WARNING syncer.py:222 -- You are using remote storage, but you don't have `fsspec` installed. This can lead to inefficient syncing behav

(DQN pid=44541) 2023-07-17 15:49:47,881	WARNING algorithm_config.py:643 -- Cannot create DQNConfig from given `config_dict`! Property __stdout_file__ not supported.
(DQN pid=44541) 2023-07-17 15:49:47,946	INFO algorithm.py:536 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(DQN pid=44541) 2023-07-17 15:49:47,954	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(DQN pid=44541) 2023-07-17 15:49:47,989	WARNING util.py:68 -- Install gputil for GPU system monitoring.


(DQN pid=44541) Loaded bsuite_id: cartpole_swingup/15.
(DQN pid=44541) Logging results to SQLite database in /Users/adamprice/RLOlympics/results/PPO/data.db.


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
DQN_cartpole_swingup_15_24889_00000,5000,"{'ObsPreprocessorConnector_ms': 0.009468623570033483, 'StateBufferConnector_ms': 0.008133479527064733, 'ViewRequirementAgentConnector_ms': 0.18689632415771484}","{'num_env_steps_sampled': 5000, 'num_env_steps_trained': 32000, 'num_agent_steps_sampled': 5000, 'num_agent_steps_trained': 32000, 'last_target_update_ts': 4501, 'num_target_updates': 8}",{},593,{},0,0,0,1,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.0002231836406281218, 'mean_q': 0.010651863180100918, 'min_q': 0.0033542783930897713, 'max_q': 0.014367694035172462, 'cur_lr': 0.0005}, 'td_error': array([-1.8708613e-03, -2.0321333e-03, 4.2565912e-04, -1.3950374e-03, 1.1789147e-05, -2.4231616e-04, -1.8147901e-03, -1.1783671e-03], dtype=float32), 'mean_td_error': -0.0010120071237906814, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 8.0, 'num_grad_updates_lifetime': 4000.0, 'diff_num_grad_updates_vs_sampler_policy': 2106.75}}, 'num_env_steps_sampled': 5000, 'num_env_steps_trained': 32000, 'num_agent_steps_sampled': 5000, 'num_agent_steps_trained': 32000, 'last_target_update_ts': 4501, 'num_target_updates': 8}",5000,32000,5000,1000,57.9062,32000,8000,463.25,0,0,0,0,8000,"{'cpu_util_percent': 17.928, 'ram_util_percent': 65.708}",{},{},{},"{'mean_raw_obs_processing_ms': 1.1539095872074483, 'mean_inference_ms': 2.1743713638976483, 'mean_action_processing_ms': 0.17567318191922615, 'mean_env_wait_ms': 0.13429271315136998, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 0.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 0.0, 'episode_len_mean': 593.0, 'episode_media': {}, 'episodes_this_iter': 1, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'episode_lengths': [985, 250, 549, 935, 363, 284, 785]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 1.1539095872074483, 'mean_inference_ms': 2.1743713638976483, 'mean_action_processing_ms': 0.17567318191922615, 'mean_env_wait_ms': 0.13429271315136998, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.009468623570033483, 'StateBufferConnector_ms': 0.008133479527064733, 'ViewRequirementAgentConnector_ms': 0.18689632415771484}}","{'training_iteration_time_ms': 17.681, 'sample_time_ms': 4.023, 'load_time_ms': 0.247, 'load_throughput': 32454.233, 'learn_time_ms': 8.463, 'learn_throughput': 945.27, 'synch_weights_time_ms': 0.035}"


2023-07-17 15:51:13,744	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-07-17 15:51:21,149	INFO tune.py:1111 -- Total run time: 107.58 seconds (100.11 seconds for the tuning loop).
2023-07-17 15:51:21,150	WARNING tune.py:1126 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: Tuner.restore(path="/Users/adamprice/ray_results/DQN", trainable=...)


ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.0002231836406281218, 'mean_q': 0.010651863180100918, 'min_q': 0.0033542783930897713, 'max_q': 0.014367694035172462, 'cur_lr': 0.0005}, 'td_error': array([-1.8708613e-03, -2.0321333e-03,  4.2565912e-04, -1.3950374e-03,
        1.1789147e-05, -2.4231616e-04, -1.8147901e-03, -1.1783671e-03],
      dtype=float32), 'mean_td_error': -0.0010120071237906814, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 8.0, 'num_grad_updates_lifetime': 4000.0, 'diff_num_grad_updates_vs_sampler_policy': 2106.75}}, 'num_env_steps_sampled': 5000, 'num_env_steps_trained': 32000, 'num_agent_steps_sampled': 5000, 'num_agent_steps_trained': 32000, 'last_target_update_ts': 4501, 'num_target_updates': 8}, 'sampler_results': {'episode_reward_max': 0.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 0.0, 'episode_len_mean': 5

In [ ]:
import pandas as pd
from typing import Callable, Mapping, NamedTuple, Sequence

class BSuiteSummary(NamedTuple):
  """Container for summary metadata for a given bsuite experiment."""
  score: Callable[[pd.DataFrame], float]
  type: str
  tags: Sequence[str]
  episode: int


def _is_finished(df: pd.DataFrame, n_min: int) -> bool:
  """Check to see if every bsuite id in the dataframe is finished."""
  # At this point we have grouped by any additional hyperparameters.
  # Check if we have run enough episodes for every id.
  max_time = df.groupby('bsuite_id')['episode'].max().reset_index()
  return max_time['episode'].min() >= n_min

def _bsuite_score_single(df: pd.DataFrame,
                         experiment_info: Mapping[str, BSuiteSummary],
                         verbose: bool = False) -> pd.DataFrame:
  """Score the bsuite across all domains for a single agent."""
  data = []
  for env_name, env_data in df.groupby('bsuite_env'):
    if env_name not in experiment_info:
      if verbose:
        print('WARNING: {}_score not found in load.py and so is excluded.'
              .format(env_name))
    else:
      b_summary = experiment_info[env_name]
      data.append({
          'bsuite_env': env_name,
          'score': b_summary.score(env_data),
          'type': b_summary.type,
          'tags': str(b_summary.tags),
          'finished': _is_finished(env_data, b_summary.episode),
      })
  return pd.DataFrame(data)

In [ ]:
import sqlite3
import pandas as pd
from bsuite.logging import sqlite_load
# Create your connection.
cnx = sqlite3.connect('/Users/adamprice/RLOlympics/results/PPO/data.db')

df = pd.read_sql_query("SELECT * FROM cartpole_swingup", cnx)

from bsuite.experiments.cartpole_swingup.analysis import score
from bsuite.experiments.summary_analysis import BSUITE_INFO

experiments = {'results/PPO/data.db'}  # Add results here
DF, SWEEP_VARS = sqlite_load.load_bsuite(experiments)


_bsuite_score_single(df, BSUITE_INFO, True)

""


In [ ]:


# Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master
WHERE type='table';"""

# Creating cursor object using connection object
cursor = cnx.cursor()
    
# executing our sql query
cursor.execute(sql_query)
print("List of tables\n")
    
# printing all tables list
print(cursor.fetchall())

List of tables

[('cartpole_swingup',), ('catch',)]
